# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
location = "output_data/cities.csv"
cities_df = pd.read_csv(location)
#len(cities_df["City"].unique())
del cities_df["Unnamed: 0"]
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Teknāf,50,BD,1586981242,87,20.86,92.31,74.97,3.42
1,Butaritari,96,KI,1586981243,81,3.07,172.79,82.90,17.90
2,Ola,92,RU,1586981243,94,59.58,151.28,20.34,3.47
3,Klaksvík,55,FO,1586981243,80,62.23,-6.59,41.00,19.46
4,Busselton,76,AU,1586981243,62,-33.65,115.33,55.00,6.22
...,...,...,...,...,...,...,...,...,...
571,Jacmel,12,HT,1586981341,66,18.23,-72.53,86.00,11.41
572,Jiangyou,9,CN,1586981342,62,31.77,104.72,61.36,2.84
573,Mukhen,0,RU,1586981342,76,48.10,136.10,35.13,4.32
574,Cullera,75,ES,1586981342,93,39.17,-0.25,61.00,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import gmaps
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]
#setting the max_intensity to max. humidity in the data set
max1 = cities_df["Humidity"].max()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity= float(max1),
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#My ideal weather
my_max_temp = cities_df.loc[(cities_df["Max Temp"] > 70) & 
                            (cities_df["Max Temp"] < 90) & 
                           (cities_df["Wind Speed"] < 10) &
                           (cities_df["Cloudiness"] == 0)]
#my_max_temp
my_max_temp.dropna(how='all')

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
38,Mocuba,0,MZ,1586981249,89,-16.84,36.99,72.82,2.84
97,Porto-Novo,0,BJ,1586981259,78,6.50,2.60,83.59,9.35
387,Saint-Pierre,0,RE,1586981309,69,-21.34,55.48,75.20,6.93
543,Vandalūr,0,IN,1586981336,79,12.89,80.08,84.20,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = my_max_temp
hotel_df['Hotel Name'] = ""
#cities= hotel_df["City"]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    location = str(latitude)+","+str(longitude)
    params = {
    "location":location, 
    "radius": 5000,
    "type": "lodging",
    "key": g_key,    
    }
    
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    print({index})
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        #print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("No hotel found/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = "No hotel found"
hotel_df

/Users/luminda/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


{38}


/Users/luminda/opt/anaconda3/envs/pythondata/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{97}
{387}
{543}


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,Mocuba,0,MZ,1586981249,89,-16.84,36.99,72.82,2.84,Phama Lodge
97,Porto-Novo,0,BJ,1586981259,78,6.50,2.60,83.59,9.35,Tour Eiffel Hotel Benin
387,Saint-Pierre,0,RE,1586981309,69,-21.34,55.48,75.20,6.93,Lindsey Hôtel
543,Vandalūr,0,IN,1586981336,79,12.89,80.08,84.20,2.24,OYO 2546 Chrompet


In [6]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,Mocuba,0,MZ,1586981249,89,-16.84,36.99,72.82,2.84,Phama Lodge
97,Porto-Novo,0,BJ,1586981259,78,6.50,2.60,83.59,9.35,Tour Eiffel Hotel Benin
387,Saint-Pierre,0,RE,1586981309,69,-21.34,55.48,75.20,6.93,Lindsey Hôtel
543,Vandalūr,0,IN,1586981336,79,12.89,80.08,84.20,2.24,OYO 2546 Chrompet


In [7]:
narrowed_city_df = hotel_df
#narrowed_city_df.dropna(axis =0, how='any')
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
38,Mocuba,0,MZ,1586981249,89,-16.84,36.99,72.82,2.84,Phama Lodge
97,Porto-Novo,0,BJ,1586981259,78,6.50,2.60,83.59,9.35,Tour Eiffel Hotel Benin
387,Saint-Pierre,0,RE,1586981309,69,-21.34,55.48,75.20,6.93,Lindsey Hôtel
543,Vandalūr,0,IN,1586981336,79,12.89,80.08,84.20,2.24,OYO 2546 Chrompet


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# Add marker layer ontop of heat map
fig
#print(locations)
final_hotels = (hotel_df["Hotel Name"]+","+ hotel_df["City"]+","+
               hotel_df["Country"]).to_list()

markers = gmaps.marker_layer(locations,hover_text=final_hotels,
                             info_box_content=final_hotels)
# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig
#final_hotels

Figure(layout=FigureLayout(height='420px'))